In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.year.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Year'] = df_input['Year'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        data = {'action': 'bf_ajax_hook',
                'type': 'make',
                'c_year': crawler_series['Year']}

        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_check = 'https://www.rainx.com/'

                resp = requests.post('https://www.rainx.com/wp-admin/admin-ajax.php',
                                     data=data,
                                     headers=get_header(),
                                     proxies=get_proxy(),
                                     timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    if html.xpath('//option/text()')[0].strip() == 'Make':
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        list_make = [text for text in html.xpath('//option/text()')[1:]]

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend([{'Year': crawler_series['Year'],
                                       'Make No': i + 1,
                                       'Make': make} for i, make in enumerate(list_make)])

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Check_Url'] = url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Year']}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Year', 'Make No'],
                                                             ascending=[False, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/2.make-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Year'],
                                                           ascending=[False],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/make_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：69

[True] - [请求1次] - [剩余68条] - [2024/11/20 08:55:41] - 2002

[True] - [请求1次] - [剩余67条] - [2024/11/20 08:55:41] - 2001

[True] - [请求1次] - [剩余66条] - [2024/11/20 08:55:41] - 2007

[True] - [请求1次] - [剩余65条] - [2024/11/20 08:55:42] - 2017

[True] - [请求1次] - [剩余64条] - [2024/11/20 08:55:42] - 2024

[True] - [请求1次] - [剩余63条] - [2024/11/20 08:55:42] - 2018

[True] - [请求1次] - [剩余62条] - [2024/11/20 08:55:42] - 2013

[True] - [请求1次] - [剩余61条] - [2024/11/20 08:55:42] - 1997

[True] - [请求1次] - [剩余60条] - [2024/11/20 08:55:42] - 1996

[True] - [请求1次] - [剩余59条] - [2024/11/20 08:55:42] - 2012

[True] - [请求1次] - [剩余58条] - [2024/11/20 08:55:42] - 2004

[True] - [请求1次] - [剩余57条] - [2024/11/20 08:55:42] - 2021

[True] - [请求1次] - [剩余56条] - [2024/11/20 08:55:42] - 2010

[True] - [请求1次] - [剩余55条] - [2024/11/20 08:55:43] - 2006

[True] - [请求1次] - [剩余54条] - [2024/11/20 08:55:43] - 2009

[True] - [请求1次] - [剩余53条] - [2024/11/20 08:55:44] - 2016

[True] - [请求1次] - [剩余52条] - [2024/11/20 08:55:44] - 2023

[True]

C:\Users\Administrator\Desktop\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
